# Set connection with GitHub

In [1]:
# ! cd /kaggle/working && rm -rf RECsys_Challenge2024

In [2]:
from kaggle_secrets import UserSecretsClient

token = UserSecretsClient().get_secret("Token")

! git clone https://{token}@github.com/madratak/RECsys_Challenge2024.git

Cloning into 'RECsys_Challenge2024'...
remote: Enumerating objects: 7232, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 7232 (delta 28), reused 0 (delta 0), pack-reused 7154 (from 5)
Receiving objects: 100% (7232/7232), 418.03 MiB | 35.08 MiB/s, done.
Resolving deltas: 100% (3909/3909), done.
Updating files: 100% (506/506), done.


In [3]:
! pip install PyGithub requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 23.4 MB/s eta 0:00:00


In [4]:
%cd /kaggle/working/RECsys_Challenge2024 
! python run_compile_all_cython.py

/kaggle/working/RECsys_Challenge2024
run_compile_all_cython: Found 11 Cython files in 5 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/usr/bin/python3'
Compiling [1/11]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorization_Cython_Epoch.c:1252:
/usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wcpp-Wcpp]8;;]
   17 | #warning "Using deprecated NumPy API, disabl

In [5]:
from Utils.notebookFunctions import *
import numpy as np
import pandas as pd
import scipy.sparse as sps
import time
import shutil
import optuna
import json
import os
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit


K_PATH = '/kaggle/working/RECsys_Challenge2024'
GH_PATH = 'TrainedModels/WithKFCV/GraphBased'

np.random.seed(42)

## Import the repository

In [6]:
repo = get_repo_from_github(token)

Repository 'RECsys_Challenge2024' found.


In [7]:
config = {
    'model': 'P3alpha',
    'metric': 'MAP',
    'n_folds': 5,
    'tune_parameters': True,
    'database_path': '/kaggle/working/history_P3alpha_MAP.db',
    'copy_prev_best_params': False,
    'tune_best_params': True,
    'save_github': True
}

Import the database where previous tuning trials have been saved.

In [8]:
try:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/history_{config["model"]}_{config["metric"]}.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

# Construction of URM and ICM matrices

In [9]:
URM_all_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_train.csv")

n_users = len(URM_all_dataframe["user_id"].unique())
n_items = len(URM_all_dataframe["item_id"].unique())

URM_all = sps.csr_matrix((URM_all_dataframe["data"].values, 
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)),
                        shape = (n_users, n_items))

URM_all

<35736x38121 sparse matrix of type '<class 'numpy.float64'>'
	with 1764607 stored elements in Compressed Sparse Row format>

In [10]:
ICM_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_ICM_metadata.csv")

n_items = len(ICM_dataframe["item_id"].unique())
n_features = len(ICM_dataframe["feature_id"].unique())

ICM_all = sps.csr_matrix((ICM_dataframe["data"].values, 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all

<38121x94331 sparse matrix of type '<class 'numpy.float64'>'
	with 2940040 stored elements in Compressed Sparse Row format>

# Training

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Data_manager.split_functions.split_train_k_folds import split_train_k_folds

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

folds = split_train_k_folds(URM_all, k=config['n_folds'])

In [12]:
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender

def objective_function_P3alpha(optuna_trial):
    
    full_hyperp = {
                   "topK": optuna_trial.suggest_int("topK", 0, 1500),
                   "alpha": optuna_trial.suggest_float("alpha", 0, 2, log=False),
                   "normalize_similarity": optuna_trial.suggest_categorical("normalize_similarity", [True, False]),
                   "implicit": optuna_trial.suggest_categorical("implicit", [True, False]),
    }            

    validation_results = []
    
    for fold_idx, (URM_train_fold, URM_validation_fold) in enumerate(folds):
    
        recommender_instance = P3alphaRecommender(URM_train_fold)
        recommender_instance.fit(**full_hyperp)
        
        evaluator = EvaluatorHoldout(URM_validation_fold, cutoff_list=[10])
        result_df, _ = evaluator.evaluateRecommender(recommender_instance)
        
        validation_results.append(result_df.loc[10]["MAP"])
    
    return np.mean(validation_results)

In [13]:
if config['tune_parameters']:
    
    optuna_study = optuna.create_study(direction='maximize', study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)

    optuna_study.optimize(objective_function_P3alpha, timeout=3600*10.5)

[I 2025-01-06 23:54:05,367] Using an existing study with name 'hyperparameters_tuning_P3alpha_MAP' instead of creating a new one.


P3alphaRecommender: Similarity column 38121 (100.0%), 1940.72 column/sec. Elapsed time 19.64 sec
EvaluatorHoldout: Ignoring 173 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35563 (100.0%) in 34.12 sec. Users per second: 1042
P3alphaRecommender: Similarity column 38121 (100.0%), 1825.00 column/sec. Elapsed time 20.89 sec
EvaluatorHoldout: Ignoring 163 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35573 (100.0%) in 31.29 sec. Users per second: 1137
P3alphaRecommender: Similarity column 38121 (100.0%), 1920.57 column/sec. Elapsed time 19.85 sec
EvaluatorHoldout: Ignoring 150 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35586 (100.0%) in 30.66 sec. Users per second: 1161
P3alphaRecommender: Similarity column 38121 (100.0%), 1901.38 column/sec. Elapsed time 20.05 sec
EvaluatorHoldout: Ignoring 167 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 3556

[I 2025-01-06 23:58:40,364] Trial 550 finished with value: 0.04783471831975102 and parameters: {'topK': 92, 'alpha': 0.2669562353513171, 'normalize_similarity': True, 'implicit': False}. Best is trial 10 with value: 0.05370249298547826.


P3alphaRecommender: Similarity column 38121 (100.0%), 1335.55 column/sec. Elapsed time 28.54 sec
EvaluatorHoldout: Ignoring 173 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35563 (100.0%) in 38.84 sec. Users per second: 916
P3alphaRecommender: Similarity column 38121 (100.0%), 1093.64 column/sec. Elapsed time 34.86 sec
EvaluatorHoldout: Ignoring 163 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35573 (100.0%) in 39.90 sec. Users per second: 892
P3alphaRecommender: Similarity column 38121 (100.0%), 1335.62 column/sec. Elapsed time 28.54 sec
EvaluatorHoldout: Ignoring 150 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35586 (100.0%) in 38.94 sec. Users per second: 914
P3alphaRecommender: Similarity column 38121 (100.0%), 1290.96 column/sec. Elapsed time 29.53 sec
EvaluatorHoldout: Ignoring 167 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35569 (

[I 2025-01-07 00:05:14,421] Trial 551 finished with value: 0.04040734818808826 and parameters: {'topK': 270, 'alpha': 0.10504632785542875, 'normalize_similarity': True, 'implicit': False}. Best is trial 10 with value: 0.05370249298547826.


...

[I 2025-01-07 10:16:19,370] Trial 676 finished with value: 0.05189782858079034 and parameters: {'topK': 24, 'alpha': 0.945589274123847, 'normalize_similarity': True, 'implicit': False}. Best is trial 595 with value: 0.053754418047188966.


P3alphaRecommender: Similarity column 38121 (100.0%), 3523.84 column/sec. Elapsed time 10.82 sec
EvaluatorHoldout: Ignoring 173 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35563 (100.0%) in 27.12 sec. Users per second: 1311
P3alphaRecommender: Similarity column 38121 (100.0%), 3518.50 column/sec. Elapsed time 10.83 sec
EvaluatorHoldout: Ignoring 163 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35573 (100.0%) in 27.12 sec. Users per second: 1312
P3alphaRecommender: Similarity column 38121 (100.0%), 3518.14 column/sec. Elapsed time 10.84 sec
EvaluatorHoldout: Ignoring 150 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35586 (100.0%) in 27.23 sec. Users per second: 1307
P3alphaRecommender: Similarity column 38121 (100.0%), 3537.71 column/sec. Elapsed time 10.78 sec
EvaluatorHoldout: Ignoring 167 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 3556

[I 2025-01-07 10:19:33,388] Trial 677 finished with value: 0.020217571485928565 and parameters: {'topK': 1, 'alpha': 0.4943603141148381, 'normalize_similarity': True, 'implicit': False}. Best is trial 595 with value: 0.053754418047188966.


P3alphaRecommender: Similarity column 38121 (100.0%), 2075.14 column/sec. Elapsed time 18.37 sec
EvaluatorHoldout: Ignoring 173 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35563 (100.0%) in 34.22 sec. Users per second: 1039
P3alphaRecommender: Similarity column 38121 (100.0%), 2081.46 column/sec. Elapsed time 18.31 sec
EvaluatorHoldout: Ignoring 163 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35573 (100.0%) in 34.03 sec. Users per second: 1045
P3alphaRecommender: Similarity column 38121 (100.0%), 2066.52 column/sec. Elapsed time 18.45 sec
EvaluatorHoldout: Ignoring 150 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35586 (100.0%) in 34.26 sec. Users per second: 1039
P3alphaRecommender: Similarity column 38121 (100.0%), 2027.37 column/sec. Elapsed time 18.80 sec
EvaluatorHoldout: Ignoring 167 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 3556

[I 2025-01-07 10:24:08,988] Trial 678 finished with value: 0.0496529085032361 and parameters: {'topK': 67, 'alpha': 0.3428088232262387, 'normalize_similarity': True, 'implicit': False}. Best is trial 595 with value: 0.053754418047188966.


## Some optuna visualizations on recommender parameters

In [14]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_slice(optuna_study)
fig.show()

In [15]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_param_importances(optuna_study)
fig.show()

## Let's train the recommender with best parameter values

In [16]:
if config['tune_best_params']:

    if config['tune_parameters']:
        best_params = optuna_study.best_trial.params
    else: 
        with open(f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/best_params_{config["model"]}_{config["metric"]}.json', 'r') as best_params_json:
            best_params = json.load(best_params_json)

    recommender_instance = P3alphaRecommender(URM_train + URM_validation)
    recommender_instance.fit(**best_params)

P3alphaRecommender: Similarity column 38121 (100.0%), 2042.53 column/sec. Elapsed time 18.66 sec


# Testing

Create the recommendations for the submission. 

In [17]:
if config['tune_best_params']:

    data_target_users_test = pd.read_csv('/kaggle/input/recommender-system-2024-challenge-polimi/data_target_users_test.csv')
    create_submission(data_target_users_test, recommender_instance, f'/kaggle/working/submission_{config["model"]}_{config["metric"]}.csv')

Submission file saved as /kaggle/working/submission_P3alpha_MAP.csv


# Save Version on GitHub 

Write or import a json file where best hyperparameters are saved. 

In [18]:
if config['tune_parameters']:
    with open(f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json', 'w') as params_file:
        json.dump(optuna_study.best_params, params_file)
        
    if config['save_github']:
        upload_file(
            f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json', 
            f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/best_params_{config["model"]}_{config["metric"]}.json', 
            f'{config["model"]}_{config["metric"]} tuning results (from kaggle notebook)',
            repo
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/'\
        f'best_params_{config["model"]}_{config["metric"]}.json', 
        f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json'
    )

File 'TrainedModels/WithKFCV/GraphBased/P3alphaRecommender/OptimizingMAP/best_params_P3alpha_MAP.json' updated successfully.


Save the history of the tuned model.

In [19]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/history_{config["model"]}_{config["metric"]}.db',
        f'Tuning {config["model"]}_{config["metric"]} db updated results (from kaggle notebook)',
        repo
    )

File 'TrainedModels/WithKFCV/GraphBased/P3alphaRecommender/OptimizingMAP/history_P3alpha_MAP.db' updated successfully.


Save the best trained model and its submission.

In [20]:
if config['save_github'] and config['tune_best_params']: 
    upload_file(
                f'/kaggle/working/submission_{config["model"]}_{config["metric"]}.csv', 
                f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/Submission/submission_{config["model"]}_{config["metric"]}.csv', 
                f'New {config["model"]}_{config["metric"]} submission (from kaggle notebook)',
                repo
            )

File 'TrainedModels/WithKFCV/GraphBased/P3alphaRecommender/OptimizingMAP/Submission/submission_P3alpha_MAP.csv' updated successfully.
